In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import matplotlib.pyplot as plt

In [ ]:
nx = 15*10
ny = 25*10

n_ens = 1
nl = 2

x = torch.linspace(0,2*torch.pi, nx+1, dtype=torch.float64)
y = torch.linspace(0,2*torch.pi, ny+1, dtype=torch.float64)
X,Y = torch.meshgrid(x,y, indexing='ij')

ut = torch.stack([torch.sin(x),torch.sin(2*x)])[None,...]
ub = torch.stack([torch.sin(x),torch.sin(2*x)])[None,...]
ul = torch.stack([torch.sin(y),torch.sin(2*y)])[None,...]
ur = torch.stack([torch.sin(y),torch.sin(2*y)])[None,...]

bounds = torch.zeros((nl,nx+1,ny+1),dtype=torch.float64)
bounds[..., :,  0,  :] = ul[0]
bounds[..., :, -1,  :] = ur[0]
bounds[..., :,  :,  0] = ut[0]
bounds[..., :,  :, -1] = ub[0]

In [ ]:
plt.imshow(bounds[0].T)
plt.colorbar()
plt.show()
plt.imshow(bounds[1].T)
plt.colorbar()
plt.show()

In [ ]:
from qgsw.solver.boundary_conditions import BilinearExtendedBoundary

boundary = BilinearExtendedBoundary()
u=boundary.compute(x,y, ut,ub,ul,ur)
lap_u = boundary.compute_laplacian(x,y, ut,ub,ul,ur)

plt.imshow(u[0,0].T)
plt.colorbar()
plt.title("Boundary field")
plt.show()

plt.imshow(lap_u[0,0].T)
plt.colorbar()
plt.title("Laplacian of the boundary field")
plt.show()

In [ ]:
from qgsw.models.qg.stretching_matrix import compute_layers_to_mode_decomposition
from qgsw.solver.helmholtz import compute_laplace_dstI, solve_helmholtz_dstI
from qgsw.spatial.core.grid_conversion import points_to_surfaces

def compute_psi_from_q_homogenous(q:torch.Tensor, f0:float, A:torch.Tensor, dx:float, dy:float) -> torch.Tensor:
    """Compute the streamfunction from potential assuming zeros along the boundary.

    Args:
        q (torch.Tensor): Potential vorticity.
            └── (..., nl, nx, ny)-shaped
        f0 (float): Coriolis parameter.
        A (torch.Tensor): Stretching matrix.
        dx (float): Infinitesimal length in the x direction.
        dy (float): Infinitesimal length in the y direction.

    Returns:
        torch.Tensor: Stream function.
            └── (..., nl, nx+1, ny+1)-shaped
    """

    nl, nx,ny = q.shape[-3:]

    Cm2l, lambd, Cl2m = compute_layers_to_mode_decomposition(A)
    lambd = lambd.reshape((1, nl, 1, 1))
    
    q_i = points_to_surfaces(q)

    rhs = torch.einsum("lm,...mxy->...lxy",Cl2m,q_i)

    laplace_dst = compute_laplace_dstI(nx,ny,dx,dy).unsqueeze(0).unsqueeze(0)

    helmholtz_dstI = laplace_dst - f0**2 * lambd


    psi_h_modes = solve_helmholtz_dstI(rhs, helmholtz_dstI)
    return torch.einsum("lm,...mxy->...lxy",Cm2l,psi_h_modes)

def compute_psi_from_q_inhomogenous(q:torch.Tensor, boundary_field:torch.Tensor,boundary_laplacian:torch.Tensor, f0:float, A:torch.Tensor, dx:float, dy:float ) -> torch.Tensor:
    """Compute the streamfunction from potential assuming zeros along the boundary.

    Args:
        q (torch.Tensor): Potential vorticity.
            └── (..., nl, nx, ny)-shaped
        boundary_field (torch.Tensor): Boundary-extrapolated field.
        boundary_laplacian (torch.Tensor): Laplacian of the boundary-extrapolated field.
        f0 (float): Coriolis parameter.
        A (torch.Tensor): Stretching matrix.
        dx (float): Infinitesimal length in the x direction.
        dy (float): Infinitesimal length in the y direction.

    Returns:
        torch.Tensor: Stream function.
            └── (..., nl, nx+1, ny+1)-shaped
    """

    psi_b = boundary_field

    q_b = points_to_surfaces(torch.nn.functional.pad(boundary_laplacian,(1,1,1,1)) - f0**2 * torch.einsum("lm,...mxy->...lxy",A,psi_b))
    q_tot = q - q_b
    psi_h = compute_psi_from_q_homogenous(q_tot,f0,A,dx,dy)
    return psi_h+psi_b

## Tests with random PV values

In [ ]:
from qgsw.models.qg.stretching_matrix import compute_A
from qgsw.physics.coriolis.beta_plane import BetaPlane

A = compute_A(torch.tensor([400,1100]),torch.tensor([9.81,0.025]), dtype=torch.float64,device="cpu")

beta_plane = BetaPlane(f0=9.375e-5,beta=1.754e-11)
f0 = beta_plane.f0

q = torch.rand((1,nl,nx,ny),dtype=torch.float64)
dx = x[1]-x[0]
dy = y[1]-y[0]
psi_h = compute_psi_from_q_homogenous(q,f0,A,dx,dy)
psi = compute_psi_from_q_inhomogenous(q,u,lap_u,f0,A,dx,dy)

In [ ]:
layer = 0

vmax = q[0,layer].abs().max()

plt.imshow(q[0,layer].T.cpu(),vmax=vmax,vmin=-vmax,cmap="RdBu_r")
plt.colorbar()
plt.title("PV")
plt.show()

vmax = psi_h[0,layer].abs().max()
plt.imshow(psi_h[0,layer].T.cpu(),vmax=vmax,vmin=-vmax,cmap="RdBu_r")
plt.colorbar()
plt.title("ѱ_h")
plt.show()
plt.imshow(psi[0,layer].T.cpu(),vmax=vmax,vmin=-vmax,cmap="RdBu_r")
plt.colorbar()
plt.title("ѱ")
plt.show()
vmax = (psi_h[0,layer]-psi[0,layer]).abs().max()
plt.imshow((psi[0,layer]-psi_h[0,layer]).T.cpu(),vmax=vmax,vmin=-vmax,cmap="RdBu_r")
plt.colorbar()
plt.title("ѱ - ѱ_h")
plt.show()

## Tests with PV from previous runs

In [ ]:
from qgsw.configs.space import SpaceConfig
from qgsw.fields.variables.physical import PotentialVorticity
from qgsw.fields.variables.tuples import UVH
from qgsw.models.qg.uvh.core import QG
from qgsw.spatial.core.discretization import SpaceDiscretization2D
from qgsw.specs import defaults
from qgsw.utils.units._units import Unit
import torch.nn.functional as F


nx,ny = 128,256
beta_plane = BetaPlane(
        f0=9.375e-5,
        beta=1.754e-11
    )
space_config = SpaceConfig(
            nx=nx,
            ny=ny,
            unit=Unit.M,
            x_min=-1_280_000,
            x_max=1_280_000,
            y_min=-2_560_000,
            y_max=2_560_000)
space_2d = SpaceDiscretization2D.from_config(space_config)

uvh = UVH.from_file(
    "../output/g5k/double_gyre_qg_long/results_step_876000.pt"
)
model = QG(
    space_2d=space_2d,
    H = torch.tensor([400, 1100, 2600],**defaults.get()),
    g_prime= torch.tensor([9.81, 0.025, 0.0125],**defaults.get()),
    beta_plane=beta_plane
)
model.dt = 3600
model.set_physical_uvh(*uvh)
pv = PotentialVorticity(model.H,model.beta_plane.f0,model.space.dx,model.space.dy).compute(model.prognostic)
pv = points_to_surfaces(F.pad(pv,(1,1,1,1)))

In [ ]:
psi_h = compute_psi_from_q_homogenous(pv,model.beta_plane.f0,model.A,model.space.dx,model.space.dy)

In [ ]:
x = torch.linspace(0,2*torch.pi, model.space.nx+1, dtype=torch.float64)
y = torch.linspace(0,2*torch.pi, model.space.ny+1, dtype=torch.float64)

ut = torch.stack([torch.sin(i*x) for i in range(1,model.space.nl+1)])[None,...]
ub = torch.stack([torch.sin(i*x) for i in range(1,model.space.nl+1)])[None,...]
ul = torch.stack([torch.sin(i*y) for i in range(1,model.space.nl+1)])[None,...]
ur = torch.stack([torch.sin(i*y) for i in range(1,model.space.nl+1)])[None,...]

boundary = BilinearExtendedBoundary()
u=boundary.compute(x,y, ut,ub,ul,ur)
lap_u = boundary.compute_laplacian(x,y, ut,ub,ul,ur)

psi = compute_psi_from_q_inhomogenous(pv,u,lap_u,model.beta_plane.f0,model.A,model.space.dx,model.space.dy)

In [ ]:
layer = 0

vmax = pv[0,layer].abs().max()

plt.imshow(pv[0,layer].T.cpu(),vmax=vmax,vmin=-vmax,cmap="RdBu_r")
plt.colorbar()
plt.title("PV")
plt.show()

vmax = psi_h[0,layer].abs().max()
plt.imshow(psi_h[0,layer].T.cpu(),vmax=vmax,vmin=-vmax,cmap="RdBu_r")
plt.colorbar()
plt.title("ѱ_h")
plt.show()
plt.imshow(psi[0,layer].T.cpu(),vmax=vmax,vmin=-vmax,cmap="RdBu_r")
plt.colorbar()
plt.title("ѱ")
plt.show()
vmax = (psi_h[0,layer]-psi[0,layer]).abs().max()
plt.imshow((psi[0,layer]-psi_h[0,layer]).T.cpu(),vmax=vmax,vmin=-vmax,cmap="RdBu_r")
plt.colorbar()
plt.title("ѱ - ѱ_h")
plt.show()